# Import module

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

Or from the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

## ImageAnalysis3 and basic modules

In [2]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
from ImageAnalysis3 import *
from ImageAnalysis3.classes import _allowed_kwds

import h5py
import ast
import pandas as pd

print(os.getpid())

59976


## Chromatin_analysis_tools etc

See **functions** in the repository for [AnalysisTool_Chromatin](../../README.md)

In [3]:
# Chromatin_analysis_tools (ATC)
# Get path for the py containing functions
import os
import sys
import importlib
module_path =r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin'
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import relevant modules
import gene_selection 
importlib.reload(gene_selection)
import gene_to_loci
importlib.reload(gene_to_loci)
import gene_activity
importlib.reload(gene_activity)
import loci_1d_features
importlib.reload(loci_1d_features)  

import atac_to_loci
importlib.reload(atac_to_loci)

<module 'atac_to_loci' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\atac_to_loci.py'>

# Define folders

In [4]:
# main folder for postanalysis
postanalysis_folder = r'L:\Shiwei\postanalysis_2024\v0'
# input files for postanalysis
input_folder = os.path.join(postanalysis_folder, 'resources_from_preprocess')

# output file to be generated
output_main_folder = os.path.join(postanalysis_folder, 'radial_position')

output_analysis_folder = os.path.join(output_main_folder, 'analysis')
output_figure_folder = os.path.join(output_main_folder, 'figures')

# make new folder if needed
make_output_folder = True

if make_output_folder and not os.path.exists(output_analysis_folder):
    os.makedirs(output_analysis_folder)
    print(f'Generating analysis folder: {output_analysis_folder}.')
elif os.path.exists(output_analysis_folder):
    print(f'Use existing analysis folder: {output_analysis_folder}.')
    
if make_output_folder and not os.path.exists(output_figure_folder):
    os.makedirs(output_figure_folder)
    print(f'Generating figure folder: {output_figure_folder}.')
elif os.path.exists(output_figure_folder):
    print(f'Use existing figure folder: {output_figure_folder}.')

Use existing analysis folder: L:\Shiwei\postanalysis_2024\v0\radial_position\analysis.
Use existing figure folder: L:\Shiwei\postanalysis_2024\v0\radial_position\figures.


# Plotting parameters

In [5]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
plt.rc('font', family='serif')
plt.rc('font', serif='Arial')

from ImageAnalysis3.figure_tools import _double_col_width, _single_col_width, _font_size, _ticklabel_size,_ticklabel_width

import seaborn as sns
sns.set_context("paper", rc={"font.size":_font_size,"axes.titlesize":_font_size+1,"axes.labelsize":_font_size})  

In [6]:
# Other required plotting parameters
_dpi = 300
_font_size = 7
_page_width = 5.5


## cell type color-codes

In [7]:
# cell labels from RNA-MERFISH and celltype prediction
selected_cell_labels = ['L2/3 IT','L4/5 IT','L5 IT','L6 IT','L5 ET','L5/6 NP','L6 CT','L6b',
                           'Sst','Pvalb','Lamp5','Sncg','Vip',
                           'Astro','Oligo','OPC','Micro','Endo','VLMC','SMC','Peri', 
                           #'other',
                          ]
# cell palette from RNA-MERFISH UMAP and stats
celltype_palette = {'Astro':'lightcoral', 
                    'Endo':'skyblue', 
                    'L2/3 IT':'gold', 
                    'L4/5 IT':'darkorange', 
                    'L5 ET':'mediumseagreen', 
                    'L5 IT':'aqua',
                    'L5/6 NP':'darkgreen',
                    'L6 CT':'brown',
                    'L6 IT':'magenta',
                    'L6b':'blue', 
                    'Lamp5':'orange', 
                    'Micro':'peachpuff',
                    'OPC':'thistle', 
                    'Oligo':'darkviolet',
                    'Peri':'sandybrown',
                    'Pvalb':'springgreen',
                    'SMC':'rosybrown',
                    'Sncg':'darkkhaki',
                    'Sst':'steelblue', 
                    'VLMC':'saddlebrown', 
                    'Vip':'red',
                    'other':'slategray'}


In [8]:
# this is the plotting order noted based on the snRNA transcriptional acitivty if needed
sorted_cellplot_order_byRNA = ['Micro', 'Oligo', 'Endo', 'OPC', 'Astro', 'Vip', 'Lamp5',
                  'L5/6 NP', 'Sst', 'Sncg', 'Pvalb', 'L4/5 IT', 'L6 CT',
                  'L6 IT', 'L6b', 'L2/3 IT', 'L5 IT', 'L5 ET']

# Load data relevant information

## load and format codebook

[merged codebook](../resources/merged_codebook.csv) as in the repository (merged for all DNA-MERFISH libraries)

In [9]:
# Load codebook 
codebook_fname = os.path.join(input_folder,'merged_codebook.csv')
codebook_df = pd.read_csv (codebook_fname, index_col=0)

# sort df by chr and chr_order
codebook_df = loci_1d_features.sort_loci_df_by_chr_order (codebook_df)
codebook_df.head()

name id  NDB_784  NDB_755  NDB_826  NDB_713  NDB_865  \
0     1:3742742-3759944  1      0.0      0.0      0.0      0.0      0.0   
1     1:6245958-6258969  2      0.0      0.0      0.0      0.0      0.0   
2     1:8740008-8759916  3      0.0      0.0      0.0      0.0      0.0   
1016  1:9627926-9637875  1      0.0      0.0      0.0      0.0      0.0   
1017  1:9799472-9811359  2      0.0      0.0      0.0      0.0      0.0   

      NDB_725  NDB_817  NDB_710  ...  NDB_479  NDB_562  NDB_608  NDB_460  \
0         0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   
1         0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   
2         0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   
1016      0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   
1017      0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   

      NDB_563  NDB_592  NDB_368  NDB_436  NDB_629  NDB_604  
0         0.0      0.0      0.0      0.0      0.0      0.0  
1         0.0      0.0      0.0      0.0      0.0      0.0  
2         0.0      0.0      0.0      0.0      0.0      0.0  
1016      0.0      0.0      0.0      0.0      0.0      0.0  
1017      0.0      1.0      0.0      0.0      0.0      0.0  

[5 rows x 228 columns]

In [10]:
# Format the chr loci name by 
# 1. changing loci name format
# 2. extract relevant information such as id, chr, chr_order, and library etc
from gene_to_loci import loci_pos_format
loci_name_list = list(map(loci_pos_format, codebook_df['name'].tolist()))
loci_name_arr = np.array(loci_name_list)

# convert to a new dataframe and set loci name as index
codebook_df = codebook_df[['name','id','chr','chr_order','library']]
codebook_df['loci_name'] = list(loci_name_arr[:,0])
codebook_df = codebook_df.set_index ('loci_name')

codebook_df.head()

name id chr  chr_order library
loci_name                                                        
chr1_3742742_3759944  1:3742742-3759944  1   1        0.0   CTP11
chr1_6245958_6258969  1:6245958-6258969  2   1        1.0   CTP11
chr1_8740008_8759916  1:8740008-8759916  3   1        2.0   CTP11
chr1_9627926_9637875  1:9627926-9637875  1   1        3.0   CTP13
chr1_9799472_9811359  1:9799472-9811359  2   1        4.0   CTP13

# Load gProfiler results


Data can be found in the repository as [gProfiler_mmusculus_data](../6_radial_position/resources/gProfiler_mmusculus_9-8-2023_12-07-00%20PM__intersections.csv)

Data is downloaded from online gProfiler tool at <https://biit.cs.ut.ee/gprofiler/gost>

In [11]:
gPro_fname = os.path.join(output_analysis_folder, r'gProfiler_mmusculus_9-8-2023_12-07-00 PM__intersections.csv')

gPro_df = pd.read_csv(gPro_fname, skiprows=17)
gPro_df 

source                              term_name     term_id  highlighted  \
0    GO:MF                        protein binding  GO:0005515         True   
1    GO:MF         cell adhesion molecule binding  GO:0050839        False   
2    GO:MF             potassium channel activity  GO:0005267         True   
3    GO:MF  monoatomic ion gated channel activity  GO:0022839        False   
4    GO:MF                 gated channel activity  GO:0022836        False   
..     ...                                    ...         ...          ...   
228  GO:CC                       receptor complex  GO:0043235         True   
229  GO:CC             cellular anatomical entity  GO:0110165        False   
230  GO:CC                     cell body membrane  GO:0044298        False   
231  GO:CC                 neuromuscular junction  GO:0031594        False   
232  GO:CC  ionotropic glutamate receptor complex  GO:0008328        False   

     adjusted_p_value  negative_log10_of_adjusted_p_value  term_size  \
0        1.393195e-08                            7.855988      10491   
1        8.292908e-06                            5.081293        292   
2        3.592536e-05                            4.444599        119   
3        1.768536e-04                            3.752386        308   
4        1.970861e-04                            3.705344        312   
..                ...                                 ...        ...   
228      3.232907e-02                            1.490407        403   
229      3.728185e-02                            1.428503      22796   
230      3.973881e-02                            1.400785         38   
231      4.503072e-02                            1.346491        101   
232      4.991442e-02                            1.301774         41   

     query_size  intersection_size  effective_domain_size  \
0            74                 59                  25177   
1            74                 10                  25177   
2            74                  7                  25177   
3            74                  9                  25177   
4            74                  9                  25177   
..          ...                ...                    ...   
228          76                  7                  27262   
229          76                 74                  27262   
230          76                  3                  27262   
231          76                  4                  27262   
232          76                  3                  27262   

                                         intersections  
0    ANK2,ANK3,ATXN1,CADM2,CADPS,CDH12,CDH18,CHRM3,...  
1    ANK3,CDH12,CDH18,CTNNA2,CTNNA3,DMD,LRRC4C,NLGN...  
2            GRIK2,HCN1,KCNB2,KCND2,KCNH7,KCNIP4,KCNQ5  
3    GABRB1,GRID2,GRIK2,HCN1,KCNB2,KCND2,KCNH7,KCNI...  
4    GABRB1,GRID2,GRIK2,HCN1,KCNB2,KCND2,KCNH7,KCNI...  
..                                                 ...  
228        ERBB4,GABRB1,GRID2,GRIK2,LRP1B,LRRTM4,NLGN1  
229  ANK2,ANK3,ATXN1,BRINP3,CADM2,CADPS,CDH12,CDH18...  
230                                  CADM2,KCNB2,KCND2  
231                              ANK3,ERBB4,NLGN1,NRG1  
232                                 GRID2,GRIK2,LRRTM4  

[233 rows x 11 columns]

# Plot results

## plot driver terms as table

In [12]:
driver_gPro_df = gPro_df[gPro_df['highlighted']==True]
driver_gPro_df['fold_enrichment'] = ((driver_gPro_df['intersection_size']/driver_gPro_df['query_size'])/
                                     (driver_gPro_df['term_size']/driver_gPro_df['effective_domain_size']))

driver_gPro_df['ID'] = driver_gPro_df.index
plot_df = driver_gPro_df[['ID','source','term_name','term_id','adjusted_p_value','fold_enrichment']]
plot_df['fold_enrichment'] = round(plot_df['fold_enrichment'],2)
plot_df = plot_df.rename(columns={"source": "Source",
                        "term_name": "Term name",
                        "term_id": "Term ID",
                        "adjusted_p_value": "Adj. p-value",
                        "fold_enrichment": "Fold enrichment",
                       })

plot_df["Adj. p-value"] = plot_df["Adj. p-value"].apply(lambda x: str("{:e}".format(x)))
plot_df["Adj. p-value"] = plot_df["Adj. p-value"].apply(lambda x: 
                                                            (x.split('e-')[0][:4]+ 'e-' + x.split('e-')[-1])
                                                           )
plot_df 

C:\Users\shiwei\AppData\Local\Temp\3\ipykernel_59976\1978925821.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_gPro_df['fold_enrichment'] = ((driver_gPro_df['intersection_size']/driver_gPro_df['query_size'])/
C:\Users\shiwei\AppData\Local\Temp\3\ipykernel_59976\1978925821.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_gPro_df['ID'] = driver_gPro_df.index
C:\Users\shiwei\AppData\Local\Temp\3\ipykernel_59976\1978925821.py:7: SettingWithCopyWarning: 
A value is trying to be set on a cop

ID Source                                          Term name  \
0      0  GO:MF                                    protein binding   
2      2  GO:MF                         potassium channel activity   
9      9  GO:MF      phosphatidylinositol-4,5-bisphosphate binding   
16    16  GO:MF                                 PDZ domain binding   
18    18  GO:MF                                calcium ion binding   
22    22  GO:MF    postsynaptic neurotransmitter receptor activity   
24    24  GO:BP                         nervous system development   
35    35  GO:BP                                 cell-cell adhesion   
43    43  GO:BP                   regulation of membrane potential   
126  126  GO:BP                       regulation of system process   
137  137  GO:BP         ligand-gated ion channel signaling pathway   
147  147  GO:BP  cell communication involved in cardiac conduction   
150  150  GO:BP             protein localization to cell periphery   
166  166  GO:BP  calcium ion-regulated exocytosis of neurotrans...   
168  168  GO:CC                                  synaptic membrane   
172  172  GO:CC                              glutamatergic synapse   
175  175  GO:CC                                  neuron projection   
191  191  GO:CC                          potassium channel complex   
220  220  GO:CC                                    catenin complex   
228  228  GO:CC                                   receptor complex   

        Term ID Adj. p-value  Fold enrichment  
0    GO:0005515     1.39e-08             1.91  
2    GO:0005267     3.59e-05            20.01  
9    GO:0005546     3.85e-03            19.11  
16   GO:0030165     1.52e-02            14.42  
18   GO:0005509     2.78e-02             4.73  
22   GO:0098960     3.57e-02            18.90  
24   GO:0007399     6.48e-20             5.90  
35   GO:0098609     5.72e-12             8.76  
43   GO:0042391     3.98e-10            12.28  
126  GO:0044057     3.18e-03             6.23  
137  GO:1990806     8.77e-03            37.71  
147  GO:0086065     1.56e-02            32.68  
150  GO:1990778     1.87e-02             7.84  
166  GO:0048791     4.84e-02            55.15  
168  GO:0097060     2.69e-24            19.23  
172  GO:0098978     4.89e-19            14.11  
175  GO:0043005     3.19e-16             6.81  
191  GO:0034705     1.09e-09            35.09  
220  GO:0016342     2.14e-02            34.71  
228  GO:0043235     3.23e-02             6.23

In [13]:
def render_mpl_table(data, col_width=1, row_height=0.2, font_size=7.,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax


fig,ax = render_mpl_table(plot_df, header_columns=0, col_width=1, row_height=0.15, font_size=7.,)
fig.savefig(os.path.join(output_figure_folder, f"gProfiler_table.pdf"),transparent=True,
            bbox_inches='tight', dpi=300)

meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped


## plot dots with signifiance

In [15]:
import seaborn as sns
fig,ax=plt.subplots(figsize=(1.8,3),dpi=200)

gPro_df['fold_enrichment'] = ((gPro_df['intersection_size']/gPro_df['query_size'])/
                                     (gPro_df['term_size']/gPro_df['effective_domain_size']))

new_top_tick = []
new_top_term = []
top_num = 5
new_term_order = []
term_order = gPro_df.index
for _i, _source in enumerate(['GO:MF','GO:BP','GO:CC']):
    sel_gPro_df = gPro_df[gPro_df['source']==_source]
    sel_gPro_df_top = sel_gPro_df[:top_num]
    print(_source, len(sel_gPro_df))
    
    sel_term_order = list(sel_gPro_df.index)
    #sel_term_order = [o+i*1 for i,o in enumerate(sel_term_order)] # space btw dots
    sel_term_order = [o+20*_i for o in sel_term_order] # space btw source

    #for i in range(0,100):
        #random.shuffle(sel_term_order)
    new_term_order.extend(sel_term_order)
    new_top_term.extend(sel_gPro_df_top['term_name'])
    new_top_tick.extend(sel_term_order[:top_num])
    

color_dict = {'GO:MF':'red','GO:BP':'orange','GO:CC':'green'}
gPro_df['plot_y_pos']=new_term_order
sns.scatterplot(data=gPro_df, y=new_term_order, x="negative_log10_of_adjusted_p_value", 
                hue="source", sizes=0.1, alpha = 0.5, palette=color_dict,
                #size="fold_enrichment", size_norm=(1, 100),
                ax=ax,legend=False, 
               )

color_list = [color_dict[i] for i in gPro_df['source']]
plt.hlines(y=gPro_df['plot_y_pos'], xmin=-1, 
           xmax=gPro_df['negative_log10_of_adjusted_p_value'], 
           linewidth=0.1, alpha =0.8,
           color=color_list,
          )

for _i, (_term,_tick) in enumerate(zip(new_top_term, new_top_tick)):
    ax.text(-26,
            _tick+10*_i,
            _term, fontsize=6.1)

    

# highlight driver
driver_gPro_df = gPro_df[gPro_df['highlighted']==True]
sns.scatterplot(data=driver_gPro_df, y='plot_y_pos', x="negative_log10_of_adjusted_p_value", 
                hue="source", sizes=0.6, alpha = 0.9, palette=color_dict,
                #size="fold_enrichment", size_norm=(1, 100),
                ax=ax,legend=False, edgecolor='black',linewidth=0.9
               )



plt.ylim([290,-30])
plt.xlim([-1,25])
ax.set_xlabel(f"-log10 (adj. p-val)", fontsize=8, rotation=0,labelpad=3,)
ax.set_ylabel(f"GO term", fontsize=8, rotation=90,labelpad=110,)
ax.tick_params('x',labelsize=7, length=2,rotation=0)
plt.yticks([])

fig.savefig(os.path.join(output_figure_folder, f"gProfiler_scatter.pdf"),transparent=True,
            bbox_inches='tight', dpi=300)

GO:MF 24
GO:BP 144
GO:CC 65


meta NOT subset; don't know how to subset; dropped
